# Hybrid Search with FastEmbed & Qdrant



## What will we do?
This notebook demonstrates the usage of Hybrid Search with FastEmbed & Qdrant. 

1. Setup: Download and install the required dependencies
2. Preview data: Load and preview the data
3. Create Sparse Embeddings: Create SPLADE++ embeddings for the data
4. Create Dense Embeddings: Create BGE-Large-en-v1.5 embeddings for the data
5. Indexing: Index the embeddings using Qdrant
6. Search: Perform Hybrid Search using FastEmbed & Qdrant
7. Ranking: Rank the search results with Reciprocal Rank Fusion (RRF)

## Setup

In order to get started, you need only two dependencies, and we'll install them next:

In [2]:
# !pip install -qU qdrant-client fastembed datasets transformers

In [3]:
import json
from typing import List

import numpy as np
import pandas as pd
from datasets import load_dataset
from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    Distance,
    NamedSparseVector,
    NamedVector,
    SparseVector,
    PointStruct,
    SearchRequest,
    SparseIndexParams,
    SparseVectorParams,
    VectorParams,
    ScoredPoint,
)
from transformers import AutoTokenizer

import fastembed
from fastembed.sparse.sparse_text_embedding import SparseEmbedding, SparseTextEmbedding
from fastembed.text.text_embedding import TextEmbedding

fastembed.__version__

'0.2.5'

In [4]:
dataset = load_dataset("tasksource/esci")
# We'll select the first 1000 examples for this demo
dataset = dataset["train"].select(range(1000))
dataset = dataset.filter(lambda x: x['product_locale'] == "us")
dataset

## Preview Data

In [ ]:
source_df = dataset.to_pandas()
df = source_df.drop_duplicates(subset=["product_text", "product_title", "product_bullet_point", "product_brand"])
df = df.dropna(subset=["product_text", "product_title", "product_bullet_point", "product_brand"])
df.head()

In [ ]:
print(f"Catalog Item Count: {len(df)}\nQueries: {len(source_df)}")

In [ ]:
df["combined_text"] = df["product_title"] + "\n" + df["product_text"] + "\n" + df["product_bullet_point"]

In [ ]:
len(df)

## Create Sparse Embeddings

In [ ]:
sparse_model_name = "prithvida/Splade_PP_en_v1"
dense_model_name = "BAAI/bge-large-en-v1.5"
# This triggers the model download
sparse_model = SparseTextEmbedding(model_name=sparse_model_name, batch_size=32)
dense_model = TextEmbedding(model_name=dense_model_name, batch_size=32)

In [ ]:
def make_sparse_embedding(texts: List[str]):
    return list(sparse_model.embed(texts))


sparse_embedding: List[SparseEmbedding] = make_sparse_embedding("Fastembed is a great library for text embeddings!")
sparse_embedding

The previous output is a SparseEmbedding object for the first document in our list.

It contains two arrays: values and indices. 
- The 'values' array represents the weights of the features (tokens) in the document.
- The 'indices' array represents the indices of these features in the model's vocabulary.

Each pair of corresponding values and indices represents a token and its weight in the document.

This is still a little abstract, so let's use the tokenizer vocab to make sense of these indices.

In [ ]:
SparseTextEmbedding.list_supported_models()

In [ ]:
def get_tokens_and_weights(sparse_embedding, model_name):
    # Find the tokenizer for the model
    for models in SparseTextEmbedding.list_supported_models():
        if models["model"] == model_name:
            tokenizer_source = models["sources"]["hf"]
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_source)
    token_weight_dict = {}
    for i in range(len(sparse_embedding.indices)):
        token = tokenizer.decode([sparse_embedding.indices[i]])
        weight = sparse_embedding.values[i]
        token_weight_dict[token] = weight

    # Sort the dictionary by weights
    token_weight_dict = dict(sorted(token_weight_dict.items(), key=lambda item: item[1], reverse=True))
    return token_weight_dict


# Test the function with the first SparseEmbedding
print(json.dumps(get_tokens_and_weights(sparse_embedding[0], sparse_model_name), indent=4))

## Create Dense Embeddings

In [ ]:
def make_dense_embedding(texts: List[str]):
    return list(dense_model.embed(texts))


dense_embedding = make_dense_embedding("Fastembed is a great library for text embeddings!")

In [ ]:
dense_embedding[0].shape

In [ ]:
product_texts = df["combined_text"].tolist()

In [ ]:
[x for x in product_texts if not isinstance(x, str)]

In [ ]:
%%time
df["sparse_embedding"] = make_sparse_embedding(product_texts)

In [ ]:
df["sparse_embedding"]

In [ ]:
%%time
df["dense_embedding"] = make_dense_embedding(product_texts)

## Indexing

In [ ]:
client = QdrantClient(":memory:")

In [ ]:
collection_name = "esci"
client.create_collection(
    collection_name,
    vectors_config={
        "text-dense": VectorParams(
            size=1024,  # OpenAI Embeddings
            distance=Distance.COSINE,
        )
    },
    sparse_vectors_config={
        "text-sparse": SparseVectorParams(
            index=SparseIndexParams(
                on_disk=False,
            )
        )
    },
)

In [ ]:
SparseVector(indices=[1, 2, 3], values=[0.1, 0.2, 0.3])

In [ ]:
def make_points(df: pd.DataFrame)-> List[PointStruct]:
    sparse_vectors = df["sparse_embedding"].tolist()
    product_texts = df["combined_text"].tolist()
    dense_vectors = df["dense_embedding"].tolist()
    rows = df.to_dict(orient="records")
    points = []
    for idx, (text, sparse_vector, dense_vector) in enumerate(zip(product_texts, sparse_vectors, dense_vectors)):
        # print(sparse_vector)
        sparse_vector = SparseVector(indices=sparse_vector.indices.tolist(), values=sparse_vector.values.tolist())
        point = PointStruct(
            id=idx,
            payload={"text": text, "product_id": rows[idx]['product_id']},  # Add any additional payload if necessary
            vector={
                "text-sparse": sparse_vector,
                "text-dense": dense_vector.tolist(),
            },
        )
        points.append(point)
    return points


points: List[PointStruct] = make_points(df)

In [ ]:
client.upsert(collection_name, points)

## Search

In [ ]:
def search(query_text: str):
    # # Compute sparse and dense vectors
    query_sparse_vectors: List[SparseEmbedding] = make_sparse_embedding(query_text)
    query_dense_vector: List[np.ndarray] = make_dense_embedding(query_text)

    search_results = client.search_batch(
        collection_name=collection_name,
        requests=[
            SearchRequest(
                vector=NamedVector(
                    name="text-dense",
                    vector=query_dense_vector[0],
                ),
                limit=10,
                with_payload=True,
            ),
            SearchRequest(
                vector=NamedSparseVector(
                    name="text-sparse",
                    vector=SparseVector(
                        indices=query_sparse_vectors[0].indices.tolist(),
                        values=query_sparse_vectors[0].values.tolist(),
                    ),
                ),
                limit=10,
                with_payload=True,
            ),
        ],
    )

    return search_results

query_text = "panasonic fans"
search_results = search(query_text)

## Ranking

In [ ]:
def rrf(rank_lists, alpha=60, default_rank=1000):
    """
    Optimized Reciprocal Rank Fusion (RRF) using NumPy for large rank lists.

    :param rank_lists: A list of rank lists. Each rank list should be a list of (item, rank) tuples.
    :param alpha: The parameter alpha used in the RRF formula. Default is 60.
    :param default_rank: The default rank assigned to items not present in a rank list. Default is 1000.
    :return: Sorted list of items based on their RRF scores.
    """
    # Consolidate all unique items from all rank lists
    all_items = set(item for rank_list in rank_lists for item, _ in rank_list)

    # Create a mapping of items to indices
    item_to_index = {item: idx for idx, item in enumerate(all_items)}

    # Initialize a matrix to hold the ranks, filled with the default rank
    rank_matrix = np.full((len(all_items), len(rank_lists)), default_rank)

    # Fill in the actual ranks from the rank lists
    for list_idx, rank_list in enumerate(rank_lists):
        for item, rank in rank_list:
            rank_matrix[item_to_index[item], list_idx] = rank

    # Calculate RRF scores using NumPy operations
    rrf_scores = np.sum(1.0 / (alpha + rank_matrix), axis=1)

    # Sort items based on RRF scores
    sorted_indices = np.argsort(-rrf_scores)  # Negative for descending order

    # Retrieve sorted items
    sorted_items = [(list(item_to_index.keys())[idx], rrf_scores[idx]) for idx in sorted_indices]

    return sorted_items

# Example usage
rank_list1 = [('A', 1), ('B', 2), ('C', 3)]
rank_list2 = [('B', 1), ('C', 2), ('D', 3)]
rank_list3 = [('A', 2), ('D', 1), ('E', 3)]

# Combine the rank lists
sorted_items = rrf([rank_list1, rank_list2, rank_list3])
sorted_items

Based on this, let's convert our sparse and dense results into rank lists. And then, we'll use the Reciprocal Rank Fusion (RRF) algorithm to combine them.

In [ ]:
def rank_list(search_result: List[ScoredPoint]):
    return [(point.id, rank+1) for rank, point in enumerate(search_result)]

dense_rank_list, sparse_rank_list = rank_list(search_results[0]), rank_list(search_results[1])

In [ ]:
rrf_rank_list = rrf([dense_rank_list, sparse_rank_list])

In [ ]:
rrf_rank_list